In [78]:
import pandas as pd
import numpy as np
import os

### 讀取檔案 應該更改file_path剩下照著跑就可以了

In [92]:
# 讀取 CSV 檔案
file_path = '/Users/andrewm1air/WYR/ispan/house/missingdata1014/house1022.csv' 
data = pd.read_csv(file_path, encoding='utf-8')

# 提取需要的欄位
new_data = data[['編號', '交易年月日', '經度', '緯度']]

# 儲存為新的 CSV 檔案
new_data.to_csv('plot-raw-data.csv', index=False, encoding='utf-8')


/var/folders/6g/zb_yk76n3gx45563jpl8swnw0000gn/T/ipykernel_32883/711469233.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='utf-8')


In [93]:
new_data.shape

(266029, 4)

### 規範經緯度
臺南市範圍 緯度23.42-22.89 經度120.656-120

In [94]:
import pandas as pd

# 讀取 CSV 檔案
df = pd.read_csv('plot-raw-data.csv')

# 將經緯度轉換為 float，無法轉換的值將變為 NaN
df['經度'] = pd.to_numeric(df['經度'], errors='coerce')
df['緯度'] = pd.to_numeric(df['緯度'], errors='coerce')

# 刪除無法轉換的行
df = df.dropna(subset=['經度', '緯度'])

# 將交易年月日轉換為 datetime 型別
df['交易年月日'] = pd.to_datetime(df['交易年月日'], format='%Y-%m-%d')

# 定義經緯度範圍
lat_min, lat_max = 22.89, 23.42
lon_min, lon_max = 120, 120.656

# 過濾數據
filtered_df = df[(df['緯度'] >= lat_min) & (df['緯度'] <= lat_max) & 
                 (df['經度'] >= lon_min) & (df['經度'] <= lon_max)]

# 排序
filtered_df = filtered_df.sort_values(by='交易年月日')

# 將過濾後的數據存回原始的 CSV 檔案
filtered_df.to_csv('DBSCANdata-1022.csv', index=False)

print("已成功更新 DBSCANdata-1022.csv，並將經緯度儲存為 float 型別")


已成功更新 DBSCANdata-1022.csv，並將經緯度儲存為 float 型別


In [95]:
filtered_df.shape

(265815, 4)

檢查一下 交易年月日型別為datetime 經緯度為float

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266029 entries, 0 to 266028
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   編號      266029 non-null  object        
 1   交易年月日   266029 non-null  datetime64[ns]
 2   經度      266029 non-null  float64       
 3   緯度      266029 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 8.1+ MB


### 經緯度轉公里
1007改
經度120.026-120.648 台南實際長度 = 0.6214經度 × 102公里/度 ≈ 62.14公里 <br> 
緯度22.884-23.408 台南實際寬度 = 0.5236緯度 × 111公里/度 ≈ 58.120公里

---
最初的方法：
經度120-120.6 實際長度 = 0.6度 × 93公里 ≈ 55.8公里 <br> 
緯度22.5-23.5 實際寬度 = 1度 × 111公里 ≈ 111公里

In [96]:
# 1007改

filtered_df = pd.read_csv('plot-raw-data.csv', encoding='utf-8')
# 對經度進行轉換
filtered_df['經度'] = (((filtered_df['經度'] - 120.026) / 0.6214)* 62.14).round(6)

# 對緯度進行轉換
filtered_df['緯度'] = (((filtered_df['緯度'] - 22.884) /0.5236)* 58.120).round(6)
filtered_df['緯度'] = (filtered_df['緯度']).round(6)

# 將過濾後的數據存回原始的 CSV 檔案
filtered_df.to_csv('DBSCANdata-1022.csv', index=False, encoding='utf-8')

print("已成功更新 DBSCANdata-1022.csv，並進行了經緯度的轉換")


已成功更新 DBSCANdata-1022.csv，並進行了經緯度的轉換


提取year

In [97]:
df = pd.read_csv('DBSCANdata-1022.csv', encoding='utf-8')

df['交易年月日'] = pd.to_datetime(df['交易年月日'], errors='coerce')  # 轉換為datetime格式
df['year'] = df['交易年月日'].dt.year.astype('Int64')  # 提取年份並轉換為整數類型
df['year'].value_counts  # 查看結果


<bound method IndexOpsMixin.value_counts of 0         2022
1         2022
2         2022
3         2022
4         2022
          ... 
266024    2024
266025    2024
266026    2024
266027    2024
266028    2024
Name: year, Length: 266029, dtype: Int64>

groupby year

In [98]:
import os



# 確保yeargroup資料夾存在
output_folder = './yeargroup-1022'
os.makedirs(output_folder, exist_ok=True)

# 提取年份
df['year'] = df['交易年月日'].dt.year.astype('Int64')  # 確保年份為整數類型

# 根據年份分組並輸出為csv
for year, group in df.groupby('year'):
    output_file = f'{output_folder}/{int(year)}-1022.csv'  # 將年份轉換為整數以避免小數點
    group.to_csv(output_file, index=False, encoding='utf-8')
    num_rows = len(group)  # 獲取該年份的行數
    print(f'{output_file} saved with {num_rows} rows.')


./yeargroup-1022/2012-1022.csv saved with 8873 rows.
./yeargroup-1022/2013-1022.csv saved with 20407 rows.
./yeargroup-1022/2014-1022.csv saved with 18440 rows.
./yeargroup-1022/2015-1022.csv saved with 17025 rows.
./yeargroup-1022/2016-1022.csv saved with 15500 rows.
./yeargroup-1022/2017-1022.csv saved with 18367 rows.
./yeargroup-1022/2018-1022.csv saved with 20775 rows.
./yeargroup-1022/2019-1022.csv saved with 25485 rows.
./yeargroup-1022/2020-1022.csv saved with 30733 rows.
./yeargroup-1022/2021-1022.csv saved with 28681 rows.
./yeargroup-1022/2022-1022.csv saved with 21625 rows.
./yeargroup-1022/2023-1022.csv saved with 22968 rows.
./yeargroup-1022/2024-1022.csv saved with 17150 rows.


查看每年資料筆數

In [87]:
# for year, group in df.groupby('year'):
#     path = f'C:\\tainan\\yeargroup\\{year}.csv'  
#     try:
#         file = pd.read_csv(path) 
#         num_rows = len(file)  
#         print(f"{year}: {num_rows} 行")
#     except FileNotFoundError:
#         print(f"{path} 文件未找到。")

### 以下為季度的切割 目前還不需要
建立儲存的資料夾 修改year_list到你想要的範圍即可

In [88]:
# years_list = list(range(2011, 2025))

# for year in years_list:
#     folder_path = f"C:\\tainan\\yeargroup\\{year}"
#     try:
#         os.makedirs(folder_path)
#         print(f"資料夾 '{folder_path}' 創建成功。")
#     except FileExistsError:
#         print(f"資料夾 '{folder_path}' 已存在。")
#     except Exception as e:
#         print(f"錯誤: {e}")

In [89]:
# season = list(range(1,5))
# for s in season:
#     startm = 3*(s-1) + 1
#     stopm = 3*s
#     print(f"第{s}季為{startm}到{stopm}月")

In [90]:
# years_list = list(range(2011, 2024))
# for year in years_list:
#     file = f'C:\\tainan\\yeargroup\\{year}.csv'
#     ygdf = pd.read_csv(file)
#     ygdf['交易年月日'] = pd.to_datetime(ygdf['交易年月日'])
#     ygdf['month'] = ygdf['交易年月日'].dt.month
#     for s in season:
#         startm = 3*(s-1) + 1
#         stopm = 3*s
#         sgroup = ygdf[(ygdf['month'] >= startm) & (ygdf['month'] <= stopm)]
#         sgroup.to_csv(f'C:\\tainan\\yeargroup\\{year}\\{year}_S{s}.csv')

In [91]:
# years_list = list(range(2011, 2024))
# season = list(range(1,5))
# for y in years_list:
#     for s in season:
#         startm = 3*(s-1) + 1
#         stopm = 3*s
#         print(f'y{y}s{s}m{startm}to{stopm}')